In [4]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium

# Load the data
data = pd.read_csv('bquxjob_7e391cf5_18f1b46b369.csv')

# Filter data to only include the latest datetime_added
latest_datetime_added = data['datetime_added'].max()
data = data[data['datetime_added'] == latest_datetime_added]

# Convert WKT columns to geometric data
data['fire_geom'] = data['fire_wkt'].apply(wkt.loads)
data['buffered_fire_geom'] = data['buffered_fire_wkt'].apply(wkt.loads)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='fire_geom')

# Initialize a Folium map at the first geometry's centroid
map_center = gdf.geometry.iloc[0].centroid
m = folium.Map(location=[map_center.y, map_center.x], zoom_start=12)

# Function to add geometries to the map
def add_to_map(geom, map_obj, color):
    folium.GeoJson(geom, style_function=lambda x, color=color: {'fillColor': color, 'color': color}).add_to(map_obj)

# Add each geometry to the map
gdf.apply(lambda x: add_to_map(x['fire_geom'], m, 'blue'), axis=1)
gdf.apply(lambda x: add_to_map(x['buffered_fire_geom'], m, 'green'), axis=1)

# Display the map
m

In [5]:
data.columns

Index(['acq_datetime', 'fire_geojson', 'buffered_fire_geojson', 'fire_wkt',
       'buffered_fire_wkt', 'datetime_added', 'fire_geom',
       'buffered_fire_geom'],
      dtype='object')